# 1. Introduction

The objective of this project is to create a generative adversarial network (GAN) to generate pictures in the style of Monet. This is part of a rolling kaggle competition with a dataset provided by kaggle. 

During this project I will review the dataset, create a GAN model, generate a submission, and present the results. 

The GAN model will be broken up into 3 main parts, the Discriminator, the Generator, and the Loss function. Together the three models will work together to generate the submission pictures. 

## 1.1 Importing Libraries

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

from kaggle_datasets import KaggleDatasets
import matplotlib.pyplot as plt
import numpy as np

try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Device:', tpu.master())
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
except:
    strategy = tf.distribute.get_strategy()
print('Number of replicas:', strategy.num_replicas_in_sync)

AUTOTUNE = tf.data.experimental.AUTOTUNE

# 2. EDA

For exploratory data analysis (EDA) on this project lets just get our data loaded first. After loading the dataset lets look a few examples to get a better idea what we are working with. 

## 2.1 Load Data

First we need to set up a way to load the data. For this lets use the TFRecord files and define some functions.  

In [ ]:
# All images are set to 256x256 rgb
size = [256, 256]
channel = 3

# function to read record and load image

def give_image(name):
    tf_format = {
        "image_name": tf.io.FixedLenFeature([], tf.string),
        "image": tf.io.FixedLenFeature([], tf.string),
        "target": tf.io.FixedLenFeature([], tf.string)
    }
    single = tf.io.parse_single_example(name, tfrecord_format)
    image = tf.image.decode_jpeg(single['image'], channels=channel)
    image = (tf.cast(image, tf.float32) / 127.5) - 1
    image = tf.reshape(image, [*size, channel])
    image = decode_image(image)
    
    return image

# load dataset
def give_data(name, labeled=True, ordered=False):
    dataset = tf.data.TFRecordDataset(name)
    dataset = dataset.map(give_image, num_parallel_calls=AUTOTUNE)
    return dataset


In [ ]:
# get path for files 
path_monet = '/kaggle/input/gan-getting-started/monet_tfrec'
path_photo = '/kaggle/input/gan-getting-started/photo_tfrec'

# load file names
names_monet = tf.io.gfile.glob(str(path_monet + '/*.tfrec')) 
names_photo = tf.io.gfile.glob(str(path_photo + '/*.tfrec')) 

In [ ]:
## load dataset

monet = give_data(names_monet, labeled=True).batch(1)
photo = give_data(names_photo, labeled=True).batch(1)

Now that the dataset is loaded lets look a few examples. 

In [ ]:
# load example image from each set
example_monet = next(iter(monet_ds))
example_photo = next(iter(photo_ds))

# plot example

plt.subplot(121)
plt.title('Photo')
plt.imshow(example_photo[0] * 0.5 + 0.5)

plt.subplot(122)
plt.title('Monet')
plt.imshow(example_monet[0] * 0.5 + 0.5)

# 3. Model 

For this project the model being created is a GAN model. The GAN is made up of multiple parts. First a generator model that generates images. A discriminator model that classify images as real of fake. For this project it will classify as Monet or NOT Monet.  

## 3.1 Generator

In [ ]:
# Generator 
def generator():
    model = keras.Sequential()

    # Encoder
    model.add(layers.Conv2D(64, kernel_size=4, strides=2, padding='same', input_shape=(256, 256, 3)))
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Conv2D(128, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(0.2))
    model.add(layers.Conv2D(256, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(0.2))

    # Decoder
    model.add(layers.Conv2DTranspose(128, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Conv2DTranspose(64, kernel_size=4, strides=2, padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Conv2DTranspose(3, kernel_size=4, strides=2, padding='same', activation='tanh'))

    return model

## 3.2 Discriminator

In [ ]:
def discriminator(input_shape=(256, 256, 3)):
    model = Sequential()

    # First convolutional block
    model.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=input_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))

    # Second convolutional block
    model.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))

    # Third convolutional block
    model.add(Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))

    # Flatten and dense layers
    model.add(Flatten())
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))

    return model

## 3.3 Loss Function

In [ ]:
# loss function
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)

# optimizer
opt = tf.keras.optimizers.Adam(1e-4)

# disc loss
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

# gen loss
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# trainable function
def make_trainable(model, trainable):
    for layer in model.layers:
        layer.trainable = trainable

## 3.4 GAN Model

In [ ]:
# Build models
generator = generator()
discriminator = discriminator()

# Compile models
discriminator.compile(optimizer= opt, loss= loss)
generator.compile(optimizer= opt, loss= loss)

gan = Sequential([
    generator,
    discriminator
])

gan.summary()

gan.compile(optimizer= opt, loss= loss)

## 3.5 Train Model

## 3.6 Submission File

# 4. Results/Analysis

# 5. Conclusion